In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
years = list(range(2023, 2020, -1))
all_matches = []


In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        team_logo_url = soup.find('img', class_='teamlogo')['src']



        try:
            team_data = matches.copy()
            team_data["crestURL"] = team_logo_url
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
        

          



In [4]:
match_df = pd.concat(all_matches)

In [5]:
match_df.columns = [c.lower() for c in match_df.columns]

In [6]:
image_team_df = match_df[["cresturl", "team"]]
image_team_df = image_team_df.drop_duplicates()


In [7]:
image_team_df.to_csv("imgurls.csv")